<a href="https://colab.research.google.com/github/andreunilux/BSP_6/blob/main/Bert_with_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install missing dependancies
!pip install transformers accelerate


#Library used for fine tuning

from torch.utils.data import DataLoader, Dataset
from transformers import Trainer, TrainingArguments

# Pandas Dataframe Library
import json
import pandas as pd
import os
from IPython.display import display
import numpy as np 

# HuggingFace Libarary
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, BertConfig, BertTokenizer
from transformers import EarlyStoppingCallback


# Model HyperParameters
current_model="bert-base-uncased"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()
os.chdir('/content/drive/MyDrive/BSP6/init_dataset_gold')   

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_data = pd.read_json('train.json', lines=True)
val_data = pd.read_json('val.json', lines=True)
test_data = pd.read_json('test.json', lines=True)

In [4]:
# load pre-trained HateBert
config = BertConfig.from_pretrained(current_model, num_labels=3)
model = BertForSequenceClassification.from_pretrained(current_model, config=config)
tokenizer = BertTokenizer.from_pretrained(current_model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [6]:
#Train Data

X = []
for context, target in zip(train_data.context, train_data.target):
  X.append(tokenizer(context, target, padding='max_length', truncation='longest_first', max_length=230))
y_train = list(train_data.label)
print(X)
X_train_tokenized = X


#val Data
V = []
for context, target in zip(val_data.context, val_data.target):
   V.append(tokenizer(context, target, padding='max_length', truncation='longest_first', max_length=230))
y_val = list(val_data.label)
V_val_tokenized = V

#test Data
T = []
for context, target in zip(test_data.context, test_data.target):
   T.append(tokenizer(context, target, padding='max_length', truncation='longest_first', max_length=230))
y_test = list(test_data.label)
T_test_tokenized = T





Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [7]:
#print the token
print(tokenizer.decode(X_train_tokenized[0].input_ids))
print(X_train_tokenized[0])
print(len(X_train_tokenized[0].input_ids))
print(y_train[0])


print(tokenizer.decode(X_train_tokenized[1].input_ids))
print(X_train_tokenized[1])
print(len(X_train_tokenized[1].input_ids))
print(y_train[1])


[CLS] the uk is fucked. [SEP] > the ~ ~ uk ~ ~ world is fucked ftfy [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [8]:
from torch.utils.data import DataLoader, RandomSampler

# Create torch dataset
#need a dictionary
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {'input_ids':torch.tensor(self.encodings[idx].input_ids),
                'attention_mask':torch.tensor(self.encodings[idx].attention_mask),
                'labels':torch.tensor(self.labels[idx])}
        
        return item

    def __len__(self):
        return len(self.encodings)

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(V_val_tokenized, y_val)
test_dataset = Dataset(T_test_tokenized, y_test)


In [9]:
train_dataset[0]

{'input_ids': tensor([  101,  1996,  2866,  2003, 21746,  1012,   102,  1028,  1996,  1066,
          1066,  2866,  1066,  1066,  2088,  2003, 21746,  3027, 12031,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



In [11]:
# Define Trainer
args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=10,               # total number of training epochs
    per_device_train_batch_size=16,   # batch size per device during training
    per_device_eval_batch_size=16,    # batch size for evaluation
    learning_rate = 8e-8,
    evaluation_strategy="steps",
    logging_steps = 100,
    load_best_model_at_end=True
)            
                  
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)



In [12]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.045200,1.014970,0.502104,0.369476,0.502104,0.346734
200,1.006000,0.970539,0.561010,0.430924,0.561010,0.483012
300,0.930900,0.956286,0.569425,0.559223,0.569425,0.503076
400,0.927900,0.941886,0.586255,0.562240,0.586255,0.556232
500,0.829200,0.983927,0.546985,0.539527,0.546985,0.539220
600,0.786900,0.955972,0.580645,0.558557,0.580645,0.563288
700,0.726100,0.986855,0.555400,0.550632,0.555400,0.544771
800,0.689900,1.026929,0.538569,0.549897,0.538569,0.543395
900,0.606400,1.040472,0.583450,0.564218,0.583450,0.562951


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=900, training_loss=0.8387356397840712, metrics={'train_runtime': 649.3783, 'train_samples_per_second': 51.203, 'train_steps_per_second': 3.203, 'total_flos': 1700596575160560.0, 'train_loss': 0.8387356397840712, 'epoch': 4.33})

In [13]:
eval=trainer.evaluate(val_dataset)

In [14]:
eval


{'eval_loss': 0.98392653465271,
 'eval_accuracy': 0.5469845722300141,
 'eval_precision': 0.5395269576507034,
 'eval_recall': 0.5469845722300141,
 'eval_f1': 0.5392201213470798,
 'eval_runtime': 9.6493,
 'eval_samples_per_second': 73.892,
 'eval_steps_per_second': 4.664,
 'epoch': 4.33}